The code can be used to scrap the companies’ finance data from finance.yahoo page. You just need to know short company code, for example from page https://finance.yahoo.com/gainers, column Symbol. And you just need to add company’s short name in companies_list list.

Financial results takes from page like:
https://finance.yahoo.com/quote/BNKXF/financials?p=BNKXF

The code also can filter the companies by their grows’ criteria – for example you want to see only such companies, which grew 10% in 2018 year and 5% in 2019 year. 


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

pd.options.display.float_format = '{:.0f}'.format

year = ['ttm/2020', '2019/2018', '2018/2017', '2017/2016']
grown = []
for i in range(4):
    grown.append(int(input('Please, enter the percentage change in Total Revenue {}:'.format(year[i]))))
    
grown_np = np.array(grown) / 100 + 1


companies_list = ['UBER',  'BNKXF' , 'CRYYF', 'PTKFF', 'BYDIF', 'CRZBY'] 

Please, enter the percentage change in Total Revenue ttm/2020:2
Please, enter the percentage change in Total Revenue 2019/2018:2
Please, enter the percentage change in Total Revenue 2018/2017:2
Please, enter the percentage change in Total Revenue 2017/2016:2


In [12]:
def company_financal_data(company):
      
    url_is = 'https://finance.yahoo.com/quote/' + company + '/financials?p=' + company
    response = requests.get(url_is)
    soup_is = BeautifulSoup(response.text, "lxml")
    features = soup_is.find_all('div', class_='D(tbr)')

    headers = []
    temp_list = []
    label_list = []
    final = []
    index = 0
    for item in features[0].find_all('div', class_='D(ib)'):
        headers.append(item.text)
    while index <= len(features)-1:
        temp = features[index].find_all('div', class_='D(tbc)')
        for line in temp:
            temp_list.append(line.text)
        final.append(temp_list)
        temp_list = []
        index+=1
    df = pd.DataFrame(final[1:])
    df.columns = headers
    df['Company'] = company
    df = df[df['Breakdown'] == 'Total Revenue']
    return(df)

In [13]:
or_is = False
for company in companies_list:
    row = company_financal_data(company)
    if not or_is:
        db = row
        or_is = True
    else:
        db = pd.concat([db, row])

for col in list(db.columns):
    db[col] = db[col].str.replace(',','')

convert_dict = {'ttm': float,
               '12/31/2019': float,
                '12/31/2018': float,
                '12/31/2017': float, 
                '12/31/2016':float
               } 
db = db.astype(convert_dict)  
db

,Breakdown,ttm,12/31/2019,12/31/2018,12/31/2017,12/31/2016,Company
0,Total Revenue,13666000,14147000,11270000,7932000,3845000,UBER
0,Total Revenue,3498270,3490370,3564376,3285593,3360310,BNKXF
0,Total Revenue,41646287,41646287,39844006,34433683,29402146,CRYYF
0,Total Revenue,23063580175,22633476361,21074306186,20182120167,19374230958,PTKFF
0,Total Revenue,61134571,53028376,41047139,38774422,36734264,BYDIF
0,Total Revenue,8878000,8929000,8653000,9276000,9318000,CRZBY


In [14]:
result = db[(db['ttm'] > db['12/31/2019'] * grown_np[0]) &
            (db['12/31/2019'] > db['12/31/2018'] * grown_np[1]) & 
            (db['12/31/2018'] > db['12/31/2017'] * grown_np[2]) &
           (db['12/31/2017'] > db['12/31/2016'] * grown_np[3])]

In [15]:
# Prints companies that meet the criteria
print("Companies:", end = " ")
x = 1
for r in result['Company']:
    print(r, end = " ")
    if x % 4 == 0 and x > 0:
        print()
    x += 1

Companies: BYDIF 